Notebook for calculating mean sea level at different stations over 1983-2001 (NOAAs epoch).

In [1]:
import pandas as pd
from cStringIO import StringIO
import datetime
from dateutil import tz
from salishsea_tools import tidetools

import requests

In [2]:

def dateparse_archive_obs(s):

    PST_tz = tz.tzoffset("PST", -28800)
    # Convert the string to a datetime object
    unaware = datetime.datetime.strptime(s, "%Y/%m/%d")
    # Add in the local time zone (Canada/Pacific)
    aware = unaware.replace(tzinfo=PST_tz)
    # Convert to UTC
    return aware.astimezone(tz.tzutc())

In [3]:
def load_daily_means(station_no, start_date, end_date):
    
    base_url = 'http://www.meds-sdmm.dfo-mpo.gc.ca/isdm-gdsi/twl-mne/inventory-inventaire/'
    form_handler = (
        'data-donnees-eng.asp?user=isdm-gdsi&region=PAC&tst=1&no='
        + str(station_no))
    sitedata = {
        'start_period': start_date,
        'end_period': end_date,
        'resolution': 'd',
        'time_zone': 'l',
    }
    data_provider = (
        'download-telecharger.asp'
        '?File=E:%5Ciusr_tmpfiles%5CTWL%5C'
        + str(station_no) + '-'+start_date + '_slev.csv'
        '&Name=' + str(station_no) + '-'+start_date+'_slev.csv')
    # Go get the data from the DFO site
    with requests.Session() as s:
        s.post(base_url + form_handler, data=sitedata)
        r = s.get(base_url + data_provider)
    # Write the data to a fake file
    fakefile = StringIO(r.content)
    # Read the fake file
    try:
        wlev_meas = pd.read_csv(
            fakefile, skiprows=7, parse_dates=[0],
            date_parser=dateparse_archive_obs)
    except pd.parser.CParserError:
        data = {'Obs_date': datetime.datetime.strptime(start_date, '%d-%b-%Y'),
                'SLEV(metres)': float('NaN')}
        wlev_meas = pd.DataFrame(data=data, index=[0])

    wlev_meas = wlev_meas.rename(
        columns={'Obs_date': 'time', 'SLEV(metres)': 'wlev'})

    return wlev_meas

In [4]:
start= '01-Jan-1983'
end = '31-Dec-2001'

# Tofino

In [5]:
num = '8615'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

2.10298324825
(6268,)


# Port Hardy

In [6]:
num = '8408'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

2.89604103321
(6775,)


# Bamfield

In [7]:
num = '8545'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

2.03468972989
(6923,)


# Point Atkinson

In [8]:
num = '7795'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

3.09516382304
(6623,)


# Campbell River

In [9]:
num = '8074'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

2.8860304756
(6497,)


# Victoria

In [10]:
num = '7120'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

1.90174178608
(6909,)


# Patricia Bay

In [11]:
num = '7277'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].shape

2.29812829043
(6914,)


# Port Renfrew

In [12]:
num = '8525'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].size
print wlev['time'][0], wlev['time'].iloc[-1]


2.01843095178
3961
1983-01-01 08:00:00+00:00 1997-04-10 08:00:00+00:00


note Port Renfrew doens't have a full dataset

#Winter Harbour 

In [13]:
num = '8735'
wlev = load_daily_means(num,start,end)
print wlev['wlev'].mean()
print wlev['wlev'].size
print wlev['time'][0], wlev['time'].iloc[-1]

2.20529405429
4642
1989-02-01 08:00:00+00:00 2001-12-31 08:00:00+00:00


Note Winter Harbour doesn't have a full dataset.

In [20]:
yr=2006
start= '31-Dec-{}'.format(yr-1)
end = '02-Jan-{}'.format(yr+1)

stns_DFO = {'PointAtkinson': 7795, 'Victoria': 7120, 'PatriciaBay': 7277, 
            'CampbellRiver':8074, 'WinterHarbour': 8735, 'Bamfield': 8545}
MSLs={'PointAtkinson': 3.095, 'Victoria': 1.902, 'PatriciaBay': 2.298, 
      'CampbellRiver':2.886, 'WinterHarbor': 2.205}

stn='WinterHarbour'
tidetools.get_dfo_wlev(stns_DFO[stn],start,end)

